# Pipeline

1. Preprocessing
2. **Data Extraction**
3. Data Exploration
4. Model

This file performs data extraction. It generates several features and returns a .pkl file containing a dataframe with said features.

# Imports

In [2]:
import pandas as pd
import numpy as np
import datetime
from collections import defaultdict

# Data extraction into new Dataframe

In [47]:
# We don't need HomeID and AwayID here, but I put it in just for exploration
results = pd.read_pickle('preprocessed_results.pkl')
df = results[['HomeID', 'AwayID', 'FTHG', 'FTAG', 'FTR']].copy()
df

,HomeID,AwayID,FTHG,FTAG,FTR
0,0,15,3,0,1
1,1,12,1,0,1
2,2,16,0,0,0
3,3,14,6,0,1
4,4,11,2,2,0
...,...,...,...,...,...
13007,97,105,2,2,0
13008,111,106,2,0,1
13009,120,109,0,2,2
13010,122,107,2,2,0


In [48]:
df = pd.read_pickle('feature_frame.pkl')

In [15]:
# Only run this if the above cell hasn't been run!!
df['Date'] = results['Date']
df['league'] = results['Div']

In [6]:
results.sample(3)

,Div,Date,HomeID,AwayID,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
1056,E1,2013-03-30,10,23,4,1,1,1,0,H,...,14,2,9,14,11,3,0,1,0,0
1377,E1,2014-02-02,10,25,2,0,1,0,0,D,...,6,2,9,14,6,5,1,2,0,0
1371,E1,2014-02-01,12,0,2,1,1,0,1,A,...,6,1,4,18,13,1,1,0,0,0


In [33]:
# "Constants"
suffixes = {'HomeID':'_home', 'AwayID':'_away'}    # suffix dict for the feature names

id_names = ['HomeID', 'AwayID']                    # list of ID columns for iteration purposes

# suffix that's appended to all league-wide features
league_suffix = '_league'

In [5]:
#slc = df['AwayID'] == 105
#df[slc][['FTR']].value_counts() / sum(slc)

# Implementing non-relational Features from the Hubáček et al. paper

__For the moment, the features are closely aligned to those implemented by Hubáček et al. That means that most features also get implemented twice, once for the home team and once for that away team. The naming convention for this approach is FEATURE_NAME_home/away.__

Example: <br>
A_WIN_PCT_home would be the historical away winning percentage of the team that plays the current game at home. <br>
H_WIN_PCT_home would be the historical home winning percentage of that same team.

That means the word after the last underscore indicates if the value of this feature represents the home team or the away team of the current(!) game.

The current approach is to prefer readability over performance which means that it can take long to add the features to the dataframe.

In [43]:
# TODO: evtl. die feature-erstellungen in Funktionen packen um sie an und aus togglen zu können
# Alle nicht-kursiven Features aus dem Hubacek Paper müssen doppelt implementiert werden, siehe oben.

## Historical strength

In [4]:
# Returns all games that were played by a specific team between date minus two years and date.
def get_historical_games(team_id, date):
    games_by_team = results[((results['HomeID'] == team_id) | (results['AwayID'] == team_id))
                           & (results['Date'] < date) & (results['Date'] > date - datetime.timedelta(730))]
    return games_by_team

In [19]:
get_historical_games(8, datetime.date(2016, 5, 20)).head(6)

,Div,Date,HomeID,AwayID,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
1527,E1,2014-08-17,8,24,2,1,1,1,0,H,...,5,6,8,11,2,6,1,2,0,0
1539,E1,2014-08-25,18,8,3,1,1,1,0,H,...,4,3,13,7,6,7,1,1,0,0
1549,E1,2014-08-31,5,8,0,3,2,0,1,A,...,1,8,14,11,7,3,0,4,0,0
1553,E1,2014-09-13,8,0,0,1,2,0,1,A,...,1,1,9,10,7,6,2,1,0,0
1565,E1,2014-09-20,15,8,3,1,1,2,1,H,...,7,5,16,11,7,5,4,1,0,0
1574,E1,2014-09-27,8,12,1,1,0,0,0,D,...,8,5,8,9,8,2,2,1,0,0


### Winning and drawing percentages

In [20]:
# Add percentages to dataframe
df['H_WIN_PCT_home'] = 0
df['H_DRAW_PCT_home'] = 0
df['A_WIN_PCT_home'] = 0
df['A_DRAW_PCT_home'] = 0
df['H_WIN_PCT_away'] = 0
df['H_DRAW_PCT_away'] = 0
df['A_WIN_PCT_away'] = 0
df['A_DRAW_PCT_away'] = 0

for row, game in results.iterrows():
    for id_name in id_names:
        team_id = game[id_name]
        game_date = game['Date']

        suffix = suffixes[id_name]
        
        hist_games = get_historical_games(team_id, game_date)
        game_count = len(hist_games)

        # total counts of home and away games in the last two years
        home_game_count = len(hist_games[hist_games['HomeID'] == team_id])
        away_game_count = len(hist_games[hist_games['AwayID'] == team_id])

        home_games_won = len(hist_games[(hist_games['HomeID'] == team_id) & (hist_games['FTR'] == 1)])
        away_games_won = len(hist_games[(hist_games['AwayID'] == team_id) & (hist_games['FTR'] == 2)])

        home_games_drawn = len(hist_games[(hist_games['HomeID'] == team_id) & (hist_games['FTR'] == 0)])
        away_games_drawn = len(hist_games[(hist_games['AwayID'] == team_id) & (hist_games['FTR'] == 0)])

        if home_game_count > 0:         # TODO should the threshold be higher to only use significant enough data?
            df.loc[row, 'H_WIN_PCT' + suffix] = home_games_won / home_game_count
            df.loc[row, 'H_DRAW_PCT' + suffix] = home_games_drawn / home_game_count
        else:
            df.loc[row, 'H_WIN_PCT' + suffix] = 0          # TODO maybe choose some standard value like 0.25 instead of 0?
            df.loc[row, 'H_DRAW_PCT' + suffix] = 0         # Overall Median/Mean could also work.

        if away_game_count > 0:
            df.loc[row, 'A_WIN_PCT' + suffix] = away_games_won / away_game_count
            df.loc[row, 'A_DRAW_PCT' + suffix] = away_games_drawn / away_game_count
        else:
            df.loc[row, 'A_WIN_PCT' + suffix] = 0          
            df.loc[row, 'A_DRAW_PCT' + suffix] = 0 

### Goal averages and standard deviations

In [23]:
# Add goal averages and deviations to dataframe
df['H_GS_AVG_home'] = 0
df['H_GC_AVG_home'] = 0
df['A_GS_AVG_home'] = 0
df['A_GC_AVG_home'] = 0
df['H_GS_AVG_away'] = 0
df['H_GC_AVG_away'] = 0
df['A_GS_AVG_away'] = 0
df['A_GC_AVG_away'] = 0

df['H_GS_STD_home'] = 0
df['H_GC_STD_home'] = 0
df['A_GS_STD_home'] = 0
df['A_GC_STD_home'] = 0
df['H_GS_STD_away'] = 0
df['H_GC_STD_away'] = 0
df['A_GS_STD_away'] = 0
df['A_GC_STD_away'] = 0

for row, game in results.iterrows():
    for id_name in id_names:
        team_id = game[id_name]
        game_date = game['Date']

        suffix = suffixes[id_name]
        
        hist_games = get_historical_games(team_id, game_date)
        game_count = len(hist_games)
        
        # calculate averages
        df.loc[row, 'H_GS_AVG' + suffix] = hist_games[hist_games['HomeID'] == team_id]['FTHG'].mean()
        df.loc[row, 'H_GC_AVG' + suffix] = hist_games[hist_games['HomeID'] == team_id]['FTAG'].mean()
        df.loc[row, 'A_GS_AVG' + suffix] = hist_games[hist_games['AwayID'] == team_id]['FTAG'].mean()
        df.loc[row, 'A_GC_AVG' + suffix] = hist_games[hist_games['AwayID'] == team_id]['FTHG'].mean()
        
        # calculate standard deviations
        df.loc[row, 'H_GS_STD' + suffix] = hist_games[hist_games['HomeID'] == team_id]['FTHG'].std()
        df.loc[row, 'H_GC_STD' + suffix] = hist_games[hist_games['HomeID'] == team_id]['FTAG'].std()
        df.loc[row, 'A_GS_STD' + suffix] = hist_games[hist_games['AwayID'] == team_id]['FTAG'].std()
        df.loc[row, 'A_GC_STD' + suffix] = hist_games[hist_games['AwayID'] == team_id]['FTHG'].std()
        
# During this process, many NaNs occur in the beginning due to insufficient data.
# We replace them with 0.
df.fillna(0, inplace=True)

In [31]:
df.sample(5)

,HomeID,AwayID,FTHG,FTAG,FTR,Date,H_WIN_PCT_home,H_DRAW_PCT_home,A_WIN_PCT_home,A_DRAW_PCT_home,...,A_GS_AVG_away,A_GC_AVG_away,H_GS_STD_home,H_GC_STD_home,A_GS_STD_home,A_GC_STD_home,H_GS_STD_away,H_GC_STD_away,A_GS_STD_away,A_GC_STD_away
6101,51,49,1,3,2,2019-03-17,0.454545,0.272727,0.416667,0.222222,...,1.500000,1.558824,1.136515,1.314978,1.769898,1.133543,1.231246,1.379319,1.212311,1.049998
7430,78,81,3,0,1,2013-11-10,0.763158,0.184211,0.625000,0.275000,...,1.552632,1.157895,1.109893,0.724004,1.347124,0.921259,1.413987,1.025008,1.288145,1.000711
6634,76,81,2,1,1,2011-10-29,0.565217,0.260870,0.043478,0.391304,...,1.391304,1.130435,0.934622,1.083473,0.886883,1.029217,1.222322,1.042572,1.117592,1.057628
7268,68,74,1,0,1,2013-04-28,0.405405,0.270270,0.078947,0.368421,...,1.324324,1.405405,1.437904,1.168917,1.010964,1.148937,1.411445,1.172225,1.055516,1.300843
12964,120,127,1,0,1,2019-04-23,0.187500,0.312500,0.235294,0.352941,...,1.228571,1.628571,0.885061,1.264911,0.951006,1.169464,1.222475,1.171080,0.972738,1.628527


## Current form (data from the last five games)

In [8]:
# df_mode indicates whether to return the df dataframe or the results dataframe
def get_last_n_games(team_id, date, n=5, df_mode=False):
    if df_mode == False:
        games_by_team = results[((results['HomeID'] == team_id) | (results['AwayID'] == team_id))
                           & (results['Date'] < date)].sort_values(by='Date', ascending=False).head(n)
    else:
        games_by_team = df[((df['HomeID'] == team_id) | (df['AwayID'] == team_id))
                           & (df['Date'] < date)].sort_values(by='Date', ascending=False).head(n)
        
    return games_by_team

In [33]:
get_last_n_games(19, datetime.date(2011, 6, 1))

,Div,Date,HomeID,AwayID,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
374,E1,2011-05-22,9,19,4,2,1,1,1,D,...,11,7,8,5,8,8,2,1,0,0
361,E1,2011-05-14,19,2,4,3,1,3,2,H,...,8,9,16,12,5,6,2,4,0,0
353,E1,2011-05-07,5,19,1,1,0,0,0,D,...,11,7,5,13,7,6,3,2,0,0
340,E1,2011-04-30,19,13,0,0,0,0,0,D,...,6,6,10,11,4,8,0,0,0,0
328,E1,2011-04-23,19,17,1,1,0,1,1,D,...,7,6,17,8,4,4,1,1,0,0


### Winning and drawing percentages

In [34]:
df['WIN_PCT_home'] = 0
df['WIN_PCT_away'] = 0
df['DRAW_PCT_home'] = 0
df['DRAW_PCT_away'] = 0

for row, game in results.iterrows():
    for id_name in id_names:
        team_id = game[id_name]
        game_date = game['Date']

        recent = get_last_n_games(team_id, game_date)
        #print(len(recent))
        # game_count could be less than five if not enough data is available; that's the only reason for this line
        game_count = len(recent)
        
        games_won = len(recent[(recent['HomeID'] == team_id) & (recent['FTR'] == 1)
                    | (recent['AwayID'] == team_id) & (recent['FTR'] == 2)])
        
        games_drawn = len(recent[recent['FTR'] == 0])
        
        suffix = suffixes[id_name]
        
        if game_count > 0:
            df.loc[row, 'WIN_PCT' + suffix] = games_won / game_count
            df.loc[row, 'DRAW_PCT' + suffix] = games_drawn / game_count
        else:
            df.loc[row, 'WIN_PCT' + suffix] = 0
            df.loc[row, 'DRAW_PCT' + suffix] = 0
            

### Goal averages and standard deviations

In [35]:
df['GS_AVG_home'] = 0
df['GC_AVG_home'] = 0
df['GS_AVG_away'] = 0
df['GC_AVG_away'] = 0

df['GS_STD_home'] = 0
df['GC_STD_home'] = 0
df['GS_STD_away'] = 0
df['GC_STD_away'] = 0

for row, game in results.iterrows():
    for id_name in id_names:
        team_id = game[id_name]
        game_date = game['Date']

        suffix = suffixes[id_name]
        
        recent = get_last_n_games(team_id, game_date)
        game_count = len(recent)
        
        if game_count > 0:    # TODO gucken ob man vielleicht höheren threshold nehmen will und dann einfach mean assigned
            df.loc[row, 'GS_AVG' + suffix] = ((recent[recent['HomeID'] == team_id]['FTHG'].sum())
                                                + (recent[recent['AwayID'] == team_id]['FTAG'].sum())) / game_count

            df.loc[row, 'GC_AVG' + suffix] = ((recent[recent['HomeID'] == team_id]['FTAG'].sum())
                                                + (recent[recent['AwayID'] == team_id]['FTHG'].sum())) / game_count
            
            df.loc[row, 'GS_STD' + suffix] = np.std(np.append(recent[recent['HomeID'] == team_id]['FTHG'].tolist(),
                                                recent[recent['AwayID'] == team_id]['FTAG'].tolist()))
        
            df.loc[row, 'GC_STD' + suffix] = np.std(np.append(recent[recent['HomeID'] == team_id]['FTAG'].tolist(),
                                                recent[recent['AwayID'] == team_id]['FTHG'].tolist()))
            
        else:
            df.loc[row, 'GS_AVG' + suffix] = 0 
            df.loc[row, 'GC_AVG' + suffix] = 0
            df.loc[row, 'GS_STD' + suffix] = 0 
            df.loc[row, 'GC_STD' + suffix] = 0

### Rest days

In [53]:
df['REST_home'] = -1
df['REST_away'] = -1

for row, game in results.iterrows():
    for id_name in id_names:
        team_id = game[id_name]
        game_date = game['Date']

        suffix = suffixes[id_name]

        recent = get_last_n_games(team_id, game_date)
        game_count = len(recent)
        
        # overall not the nicest solution, but works
        if game_count > 0:
            last_game = get_last_n_games(team_id, game_date).head(1)     
            try:
                rest_days = (game_date - last_game.iloc[0, 1]).days      # 0 is row, 1 is column (date)
            except:
                rest_days = -1
            
            if rest_days < 0 or rest_days > 21:             # manipulate outliers
                rest_days = 7
            
            df.loc[row, 'REST' + suffix] = rest_days
        else:
            df.loc[row, 'REST' + suffix] = 7    # if now data is available, just one week of rest

## League averages

In [54]:
def get_league_games(league, date):
    games = results[(results['Div'] == league) & (results['Date'] < date) & (results['Date'] > date - datetime.timedelta(730))]
    return games

In [55]:
df['H_GS_AVG' + league_suffix] = 0
df['A_GS_AVG' + league_suffix] = 0
df['H_GS_STD' + league_suffix] = 0
df['A_GS_STD' + league_suffix] = 0
df['H_WIN_PCT' + league_suffix] = 0
df['DRAW_PCT' + league_suffix] = 0

for row, game in results.iterrows():
    league_name = game['Div']
    game_date = game['Date']

    league_games = get_league_games(league_name, game_date)
    game_count = len(league_games)

    # goals scored means
    df.loc[row, 'H_GS_AVG' + league_suffix] = league_games[league_games['Div'] == league_name]['FTHG'].mean()
    df.loc[row, 'A_GS_AVG' + league_suffix] = league_games[league_games['Div'] == league_name]['FTAG'].mean()
    
    # goals scored standard deviations
    df.loc[row, 'H_GS_STD' + league_suffix] = league_games[league_games['Div'] == league_name]['FTHG'].std()
    df.loc[row, 'A_GS_STD' + league_suffix] = league_games[league_games['Div'] == league_name]['FTAG'].std()
   
    home_games_won = len(league_games[(league_games['Div'] == league_name) & (league_games['FTR'] == 1)])
    games_drawn = len(league_games[(league_games['Div'] == league_name) & (league_games['FTR'] == 0)])
   
    if game_count > 0:
        df.loc[row, 'H_WIN_PCT' + league_suffix] = home_games_won / game_count
        df.loc[row, 'DRAW_PCT' + league_suffix] = games_drawn / game_count
    else:
        df.loc[row, 'H_WIN_PCT' + league_suffix] = 0
        df.loc[row, 'DRAW_PCT' + league_suffix] = 0

### Other league features

In [60]:
df['TEAM_CNT' + league_suffix] = 0
df['GD_STD' + league_suffix] = 0
df['RND_CNT' + league_suffix] = 0
for row, game in results.iterrows():
    league_name = game['Div']
    game_date = game['Date']
    
    # all leagues consist of 20 teams, except the Bundesliga (D1)
    if league_name == 'D1':
        df.loc[row, 'TEAM_CNT' + league_suffix] = 18
        df.loc[row, 'RND_CNT' + league_suffix] = 34
    else:
        df.loc[row, 'TEAM_CNT' + league_suffix] = 20
        df.loc[row, 'RND_CNT' + league_suffix] = 38
        
    games = get_league_games(league_name, game_date)
    
    # goal difference standard deviation
    df.loc[row, 'GD_STD' + league_suffix] = abs(games['FTHG'] - games['FTAG']).std() # muss hier wirklich abs() hin?

## Pi-ratings

### Functions

In [61]:
# Constant and error function (called psi) from the pi-ratings paper by Constantinou and Fenton
c = 3
def psi(error):
    return c * np.log10(1 + error)

def psi_h(error, exp_gd, actual_gd):
    if (exp_gd - actual_gd) < 0:
        return psi(error)
    else:
        return -1 * psi(error)

def psi_a(error, exp_gd, actual_gd):
    if (exp_gd - actual_gd) > 0:
        return psi(error)
    else:
        return -1 * psi(error)
    

# Constant and goal difference prediction function
b = 10
def predict_gd(rating):
    if rating >= 0:
        return np.power(b, np.abs(rating) / c) - 1
    else:
        return -(np.power(b, np.abs(rating) / c) - 1)

'''
Returns a dict with keys H_RTG_home, H_RTG_away, A_RTG_away, A_RTG_away
Used for one game!

cur_ratings: input in the same form as the output (dict with 4 elems); current ratings
'''
def update_pi_ratings(cur_ratings, actual_gd):
    H_RTG_home, H_RTG_away, A_RTG_home, A_RTG_away = cur_ratings['H_RTG_home'], cur_ratings['H_RTG_away'], cur_ratings['A_RTG_home'], cur_ratings['A_RTG_away']
    
    exp_gd = predict_gd(H_RTG_home) - predict_gd(A_RTG_away)    # Calculate expected goal difference
    
    error = np.abs(actual_gd - exp_gd)
    
    # The two learning rates from the pi-ratings paper (may have to be adjusted)
    lr_lambda = 0.1
    lr_gamma = 0.3
    
    # Update the pi-ratings
    new_H_RTG_home = H_RTG_home + psi_h(error, exp_gd, actual_gd) * lr_lambda 
    new_H_RTG_away = H_RTG_away + (new_H_RTG_home - H_RTG_home) * lr_gamma
    
    new_A_RTG_away = A_RTG_away + psi_a(error, exp_gd, actual_gd) * lr_lambda
    new_A_RTG_home = A_RTG_home + (new_A_RTG_away - A_RTG_away) * lr_gamma
    
    new_ratings = {'H_RTG_home': new_H_RTG_home, 'H_RTG_away': new_H_RTG_away, 'A_RTG_away': new_A_RTG_away, 'A_RTG_home': new_A_RTG_home}
    
    return new_ratings

In [62]:
update_pi_ratings({'H_RTG_home': 1.6, 'H_RTG_away': 0.4, 'A_RTG_home': 0.3, 'A_RTG_away': -1.2}, 3)

{'H_RTG_home': 1.5145736713398419,
 'H_RTG_away': 0.37437210140195254,
 'A_RTG_away': -1.1145736713398415,
 'A_RTG_home': 0.3256278985980475}

In [63]:
predict_gd(-1.5)

-2.1622776601683795

### Implementation into dataframe

In [64]:
# Returns the ID of the next game of a given team.
def get_next_game_id(team_id, date):
    games = results[((results['HomeID'] == team_id) | (results['AwayID'] == team_id))
                           & (results['Date'] > date)].sort_values(by='Date')
    
    if len(games) > 0:
        return games.index[0]    # [0] refers to the first element of the array returned by .index here
    else:
        print("LAST GAME:", date, team_id)
        return None

In [66]:
get_next_game_id(31, datetime.date(2017, 4, 2))

2576

In [ ]:
df['H_RTG_home'] = 0
df['A_RTG_home'] = 0
df['H_RTG_away'] = 0
df['A_RTG_away'] = 0

for row, game in results.iterrows():
    league_name = game['Div']
    game_date = game['Date']    
    
    # Get current pi ratings from dataframe
    cur_ratings = {
        'H_RTG_home': df.loc[row, 'H_RTG_home'],
        'H_RTG_away': df.loc[row, 'H_RTG_away'],
        'A_RTG_home': df.loc[row, 'A_RTG_home'],
        'A_RTG_away': df.loc[row, 'A_RTG_away']
    }
    
    actual_gd = game['FTHG'] - game['FTAG']
    
    new_ratings = update_pi_ratings(cur_ratings, actual_gd)
    
    # Update all pi ratings for the next game (if it wasn't a teams last game)

    next_game_id_H = get_next_game_id(game['HomeID'], game_date)
    next_game_id_A = get_next_game_id(game['AwayID'], game_date)
    
    if next_game_id_H != None:
        df.loc[next_game_id_H, 'H_RTG_home'] = new_ratings['H_RTG_home']
        df.loc[next_game_id_H, 'H_RTG_away'] = new_ratings['H_RTG_away']
    
    if next_game_id_A != None:
        df.loc[next_game_id_A, 'A_RTG_home'] = new_ratings['A_RTG_home']
        df.loc[next_game_id_A, 'A_RTG_away'] = new_ratings['A_RTG_away']
    
    # TODO: Gucken ob die Indizes passen oder ob da kleine errors drin sind die alles durcheinander wirbeln
    

### Expected goal difference for the match

In [70]:
for row, game in results.iterrows():
    exp_gd_match = predict_gd(df.loc[row, 'H_RTG_home']) - predict_gd(df.loc[row, 'A_RTG_away'])
    df.loc[row, 'EGD'] = exp_gd_match

## PageRank

To save computation time, we don't calculate an entire matrix every time. Instead, we just calculate the average points of the two given teams against each other from the last two years (like described in the Hubacek paper). 

In [20]:
# We initialize it with 1 point per game as the default value.
df['EPTS_PR_home'] = 1
df['EPTS_PR_away'] = 1

for row, game in results.iterrows():
    home_id = game['HomeID']
    away_id = game['AwayID']
    
    game_date = game['Date']

    hist_games = get_historical_games(home_id, game_date)  # Could also use away_id here, it doesn't matter

    # Previous matches of the two current teams
    previous_matches = hist_games[((home_id == hist_games['HomeID']) | (home_id == hist_games['AwayID']))
              & ((away_id == hist_games['HomeID']) | (away_id == hist_games['AwayID']))]
    
    game_count = len(previous_matches)
    
    if game_count == 0:
        continue              # Avoid division by zero by just letting the PR for both teams be 1
    
    # Number of wins of the current(!) home team in previous_matches
    win_count_H = len(previous_matches[((previous_matches['HomeID'] == home_id) & previous_matches['FTR'] == 1)
                                      |((previous_matches['AwayID'] == home_id) & previous_matches['FTR'] == 2)])
    
    # Number of draws in previous_matches
    draw_count = len(previous_matches[previous_matches['FTR'] == 0])
    
    # Number of wins of the current(!) away team in previous_matches
    win_count_A = len(previous_matches[((previous_matches['HomeID'] == away_id) & previous_matches['FTR'] == 1)
                                      |((previous_matches['AwayID'] == away_id) & previous_matches['FTR'] == 2)])
    
    df.loc[row, 'EPTS_PR_home'] = (3 * win_count_H + draw_count) / game_count
    df.loc[row, 'EPTS_PR_away'] = (3 * win_count_A + draw_count) / game_count

# Implementing own features

**In the following section, some of my own feature ideas are implemented into the dataframe.**

These can be based on historical data, current form data, league data or whatever else.

## Red Cards

Here, we accumulate the red cards each team collected in the last three games. Missing a player due to a red card can be a potentially important factor.

In [71]:
df['RED_home'] = 0
df['RED_away'] = 0

for row, game in results.iterrows():
    for id_name in id_names:
        team_id = game[id_name]
        game_date = game['Date']
        
        recent = get_last_n_games(team_id, game_date, n=3)
        
        suffix = suffixes[id_name]
        
        df.loc[row, 'RED' + suffix] = recent['HR'].sum()

## Difference between own shots and shots conceded (historical)

In [72]:
df['H_ST_home'] = 0
df['A_ST_home'] = 0
df['H_ST_away'] = 0
df['A_ST_away'] = 0

for row, game in results.iterrows():
    for id_name in id_names:
        team_id = game[id_name]
        game_date = game['Date']

        hist_games = get_historical_games(team_id, game_date)
        
        home_shots_mean = hist_games[hist_games['HomeID'] == team_id]['HS'].mean()
        away_shots_mean = hist_games[hist_games['AwayID'] == team_id]['AS'].mean()
        
        if np.isnan(home_shots_mean):
            home_shots_mean = 0       # TODO: hier besseren filler value suchen
            
        if np.isnan(away_shots_mean):
            away_shots_mean = 0       # TODO: hier besseren filler value suchen
            
        df.loc[row, 'H_ST' + suffixes[id_name]] = home_shots_mean
        df.loc[row, 'A_ST' + suffixes[id_name]] = away_shots_mean

## Match importance (similar to Hubacek)

### Season

In [7]:
# Returns the season corresponding to a certain date.
def get_season_from_date(date):
    for i in range(9):
        if date < datetime.date(2010 + i+1, 7, 15):
            return i+1

In [74]:
# To build up tables, there needs to be a cut where one season ends and the next begins.
# This is the 15th of July every year here.
df['season'] = 0

for row, game in results.iterrows():
    game_date = game['Date']
    #print(get_season_from_date(game_date))
    df.loc[row, 'season'] = get_season_from_date(game_date)

### Round (1-34 / 1-38)

In [27]:
df['RND'] = 0

prev_season = 1
cur_matchday = 1

modulo_add = 1
idx_add = 0

for i in range(len(df.index)):
    game_season = df.loc[i, 'season']

    # This fixes the matchday for games that were postponed
    if i == 6969:
        idx_add += 1
    if i == 7931:
        idx_add += 1
    if i == 7939:
        idx_add += 1
    if i == 7981:
        df.loc[i, 'RND'] = 24
        idx_add -= 1
        continue
    if i == 7992:
        df.loc[i, 'RND'] = 25
        idx_add -= 1
        continue
    if i == 8467:
        idx_add += 1
    if i == 8474:
        idx_add += 1
    if i == 8611:
        df.loc[i, 'RND'] = 3
        idx_add -= 1
        continue
    if i == 157:
        idx_add += 1
    if i == 231:
        df.loc[i, 'RND'] = 16
        idx_add -= 1
        continue
        
    if game_season != prev_season:       # check if new season has begun
        cur_matchday = 1
        #idx_add = 0

    matches_per_md = int(np.round(df.loc[i, 'TEAM_CNT_league'] / 2))

    df.loc[i, 'RND'] = cur_matchday

    # Change modulo_add for the switch from Germany (9 games per matchday) to Italy (10 games per matchday).
    # Used to avoid getting bugs in current matchday.
    if i == 6174:
        modulo_add = -3
        
    if (i + idx_add + modulo_add) % matches_per_md == 0:      # check if last game of a matchday has been played
        cur_matchday += 1

    prev_season = game_season

In [32]:
df[210:250]

,HomeID,AwayID,FTHG,FTAG,FTR,Date,H_WIN_PCT_home,H_DRAW_PCT_home,A_WIN_PCT_home,A_DRAW_PCT_home,...,RED_away,H_ST_home,A_ST_home,H_ST_away,A_ST_away,season,RND,league,PTS_AFTER_GAME_home,PTS_AFTER_GAME_away
210,19,8,2,1,1,2011-01-12,0.285714,0.285714,0.416667,0.166667,...,0,15.000000,10.500000,13.200000,11.800000,1,22,E1,28,25
211,3,1,2,0,1,2011-01-15,0.700000,0.200000,0.272727,0.272727,...,1,16.000000,14.818182,11.090909,8.181818,1,22,E1,38,28
212,18,7,4,3,1,2011-01-15,0.545455,0.272727,0.545455,0.272727,...,0,13.727273,10.545455,10.363636,9.100000,1,22,E1,45,21
213,13,2,2,0,1,2011-01-15,0.454545,0.181818,0.300000,0.100000,...,0,14.000000,8.800000,14.727273,9.909091,1,22,E1,30,30
214,14,19,3,2,1,2011-01-15,0.400000,0.200000,0.181818,0.181818,...,0,15.900000,11.000000,14.750000,10.500000,1,22,E1,25,28
215,15,10,0,3,2,2011-01-15,0.272727,0.363636,0.090909,0.363636,...,2,13.090909,10.090909,17.200000,14.090909,1,22,E1,20,43
216,6,16,1,1,0,2011-01-15,0.181818,0.454545,0.200000,0.400000,...,0,13.363636,11.000000,12.727273,10.100000,1,22,E1,22,23
217,11,0,1,1,0,2011-01-16,0.300000,0.500000,0.100000,0.500000,...,1,9.000000,8.400000,13.000000,10.800000,1,22,E1,23,22
218,8,12,2,2,0,2011-01-16,0.600000,0.200000,0.090909,0.181818,...,0,13.200000,11.636364,15.400000,13.000000,1,22,E1,26,26
219,4,17,1,1,0,2011-01-16,0.545455,0.363636,0.181818,0.454545,...,1,14.818182,11.000000,12.545455,9.700000,1,23,E1,34,29


### Current standings

In [32]:
get_last_n_games(8, datetime.date(2010,8,22), n=1, df_mode=True)

,HomeID,AwayID,FTHG,FTAG,FTR,H_WIN_PCT_home,H_DRAW_PCT_home,A_WIN_PCT_home,A_DRAW_PCT_home,H_WIN_PCT_away,...,RED_away,H_ST_home,H_ST_away,A_ST_home,A_ST_away,season,RND,Date,PTS_AFTER_GAME_home,PTS_AFTER_GAME_away
8,8.0,10.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2010-08-15,1,1


In [49]:
# Points after the current game
df['PTS_AFTER_GAME_home'] = 0
df['PTS_AFTER_GAME_away'] = 0

# Number of games after the current game
df['GAME_CNT_AFTER_GAME_home'] = 0
df['GAME_CNT_AFTER_GAME_away'] = 0

for row, game in df.iterrows():
    #print(row)
    # Game result
    game_res = game['FTR']
    game_date = game['Date']
    
    # Get previous points and game_cnt of both teams (0 if this game is part of the first matchday)
    if game['RND'] == 1:
        prev_points_home = 0
        prev_points_away = 0
        
        prev_game_cnt_home = 0
        prev_game_cnt_away = 0
    else:
        # Last game played by the home team
        last_game_home = get_last_n_games(game['HomeID'], game_date, n=1, df_mode=True)
        # Last game played by the away team
        last_game_away = get_last_n_games(game['AwayID'], game_date, n=1, df_mode=True)
        
        #print(last_game_home.iloc[0])
        #print(row)
        
        # Get points from after the teams' last games
        if game['HomeID'] == last_game_home.iloc[0]['HomeID']:
            prev_points_home = last_game_home.iloc[0]['PTS_AFTER_GAME_home']
            prev_game_cnt_home = last_game_home.iloc[0]['GAME_CNT_AFTER_GAME_home']
        elif game['HomeID'] == last_game_home.iloc[0]['AwayID']:
            prev_points_home = last_game_home.iloc[0]['PTS_AFTER_GAME_away']
            prev_game_cnt_home = last_game_home.iloc[0]['GAME_CNT_AFTER_GAME_away']
        
        if game['AwayID'] == last_game_away.iloc[0]['HomeID']:
            prev_points_away = last_game_away.iloc[0]['PTS_AFTER_GAME_home']
            prev_game_cnt_away = last_game_away.iloc[0]['GAME_CNT_AFTER_GAME_home']
        elif game['AwayID'] == last_game_away.iloc[0]['AwayID']:
            prev_points_away = last_game_away.iloc[0]['PTS_AFTER_GAME_away']
            prev_game_cnt_away = last_game_away.iloc[0]['GAME_CNT_AFTER_GAME_away']
     
    # Fix wrong stuff
    if prev_game_cnt_home + 1 > int(df.loc[row, 'RND_CNT_league']):
        prev_game_cnt_home -= int(df.loc[row, 'RND_CNT_league'])
        prev_points_home = 0
    if prev_game_cnt_away + 1 > int(df.loc[row, 'RND_CNT_league']):
        prev_game_cnt_away -= int(df.loc[row, 'RND_CNT_league'])
        prev_points_away = 0
    
    if game_res == 1:      # Home win
        df.loc[row, 'PTS_AFTER_GAME_home'] = prev_points_home + 3
        df.loc[row, 'PTS_AFTER_GAME_away'] = prev_points_away
    elif game_res == 0:    # Draw
        df.loc[row, 'PTS_AFTER_GAME_home'] = prev_points_home + 1
        df.loc[row, 'PTS_AFTER_GAME_away'] = prev_points_away + 1
    elif game_res == 2:    # Away win
        df.loc[row, 'PTS_AFTER_GAME_away'] = prev_points_away + 3
        df.loc[row, 'PTS_AFTER_GAME_home'] = prev_points_home
    
    
    
    df.loc[row, 'GAME_CNT_AFTER_GAME_home'] = prev_game_cnt_home + 1
    df.loc[row, 'GAME_CNT_AFTER_GAME_away'] = prev_game_cnt_away + 1

In [50]:
df[9470-260:9490-250]

,HomeID,AwayID,FTHG,FTAG,FTR,Date,H_WIN_PCT_home,H_DRAW_PCT_home,A_WIN_PCT_home,A_DRAW_PCT_home,...,A_ST_home,H_ST_away,A_ST_away,season,RND,league,PTS_AFTER_GAME_home,PTS_AFTER_GAME_away,GAME_CNT_AFTER_GAME_home,GAME_CNT_AFTER_GAME_away
9210,95,70,3,1,1,2018-05-20,0.222222,0.444444,0.157895,0.315789,...,8.947368,13.763158,12.027027,8,38,I1,38,54,38,38
9211,63,71,1,0,1,2018-05-20,0.351351,0.189189,0.263158,0.157895,...,10.526316,11.289474,9.135135,8,38,I1,40,39,38,38
9212,65,78,2,3,2,2018-08-18,0.342105,0.289474,0.236842,0.157895,...,10.368421,16.631579,13.289474,9,1,I1,0,3,1,1
9213,79,81,1,2,2,2018-08-18,0.552632,0.184211,0.552632,0.236842,...,13.368421,19.657895,15.131579,9,1,I1,0,3,1,1
9214,71,95,0,1,2,2018-08-19,0.368421,0.157895,0.210526,0.210526,...,9.105263,10.421053,8.947368,9,1,I1,0,3,1,1
9215,90,72,2,0,1,2018-08-19,0.263158,0.210526,0.157895,0.210526,...,9.000000,12.315789,8.526316,9,1,I1,3,0,1,1
9216,69,63,2,2,0,2018-08-19,0.000000,0.000000,0.000000,0.000000,...,0.000000,12.578947,10.526316,9,1,I1,1,1,1,1
9217,89,74,1,0,1,2018-08-19,0.297297,0.270270,0.342105,0.184211,...,11.868421,17.289474,14.289474,9,1,I1,3,0,1,1
9218,86,62,0,1,2,2018-08-19,0.447368,0.368421,0.236842,0.394737,...,11.789474,19.394737,15.342105,9,1,I1,0,3,1,1
9219,82,91,4,0,1,2018-08-20,0.552632,0.263158,0.421053,0.289474,...,13.315789,0.000000,0.000000,9,1,I1,3,0,1,1


In [119]:
#df[6950:7000]
#df[6960+330:7000+330]

In [184]:
#df[9190-730:9230-730]
#df[8470+350:8490+360]

In [185]:
df[df['RND'] == 39]

,HomeID,AwayID,FTHG,FTAG,FTR,Date,H_WIN_PCT_home,H_DRAW_PCT_home,A_WIN_PCT_home,A_DRAW_PCT_home,...,RED_away,H_ST_home,A_ST_home,H_ST_away,A_ST_away,season,RND,league,PTS_AFTER_GAME_home,PTS_AFTER_GAME_away


In [21]:
# Calculate table by points from of last matchday

In [20]:
# Gets current position of a team AFTER a given matchday in a given season
# TODO: perhaps remove team id
def get_cur_standings_old(team_id, season, matchday, league):
    # Firstly get all the games on that matchday
    games_on_md = df[(df['season'] == season) & (df['RND'] == matchday) & (df['league'] == league)]
    
    cur_standings = pd.DataFrame(columns=['team_id', 'pts'])
    
    for i in [x * 2 for x in list(range(len(games_on_md)))]:
        cur_standings.loc[i, 'team_id'] = games_on_md['HomeID'].iloc[int(i/2)]
        cur_standings.loc[i, 'pts'] = games_on_md['PTS_AFTER_GAME_home'].iloc[int(i/2)]
        cur_standings.loc[i+1, 'team_id'] = games_on_md['AwayID'].iloc[int(i/2)]
        cur_standings.loc[i+1, 'pts'] = games_on_md['PTS_AFTER_GAME_away'].iloc[int(i/2)]
    
    cur_standings.sort_values(by='pts', axis=0, ascending=False, inplace=True)
    
    cur_standings.index = range(1, len(cur_standings.index) + 1)
    
    for elem in cur_standings.duplicated(subset=['team_id']):
        if elem == True:
            print(f'season: {season}, md: {matchday}, league: {league}')
    
    return cur_standings

In [51]:
# The idea here is to disregard the matchday values in RND and just go by taking every team's points after its last game.
# That way we are relatively independent from postponed games.
# This shall be done by points PER GAME(!)
def get_cur_standings(season, league, idx):
    if league == 'D1':
        team_cnt_league = 18
    else:
        team_cnt_league = 20
    
    season_games = df[(df['season'] == season) & (df['league'] == league)]
    
    first_index = int(season_games.iloc[0].name)
    
    cur_standings = pd.DataFrame(columns=['team_id', 'pts'])
    
    for i in range(idx):
        if idx-i < first_index:
            break

        game = season_games.loc[idx-i]
        
        # Das hier sollte eigentlich funktionieren. Vielleicht irgendwelche probleme mit datentyp?
        if game['HomeID'] not in cur_standings['team_id'].values:
            #print(game['HomeID'], "\n", cur_standings[cur_standings['team_id'] == game['HomeID']])
            cur_standings.loc[len(cur_standings), 'team_id'] = game['HomeID']
            cur_standings.loc[len(cur_standings)-1, 'pts'] = int(game['PTS_AFTER_GAME_home']) / int(game['GAME_CNT_AFTER_GAME_home'])
            
        if game['AwayID'] not in cur_standings['team_id'].values:
            cur_standings.loc[len(cur_standings), 'team_id'] = game['AwayID']
            cur_standings.loc[len(cur_standings)-1, 'pts'] = int(game['PTS_AFTER_GAME_away']) / int(game['GAME_CNT_AFTER_GAME_away'])
    
    cur_standings.sort_values(by='pts', axis=0, ascending=False, inplace=True)
    
    cur_standings.index = range(1, len(cur_standings.index) + 1)
    
    return cur_standings

In [31]:
get_cur_standings(9, 24, 'D1', 6083)

,team_id,pts
1,35,2.25
2,42,2.25
3,61,1.875
4,41,1.79167
5,45,1.66667
6,51,1.625
7,50,1.625
8,54,1.45833
9,40,1.41667
10,49,1.375


In [ ]:
get_cur_standings(5, 38, 'I1')

In [25]:
get_cur_standings_old(69, 5, 38, 'I1')

,team_id,pts
1,78,87
2,62,70
3,79,69
4,66,64
5,81,63
6,77,59
7,70,56
8,74,55
9,86,54
10,67,52


### Relative point differences

In [35]:
# Return normalized (per game) points difference between a team and a given position, given a standings table
# (Used for the match importance feature)

# - team_id: id of the team that should be compared to a position
# - standings: table generated by get_cur_standings
# - pos: position that team_id should be compared to
# - match_count: number of matchdays that were played before. used to calculate difference per game

def get_rel_pts_diff_old(team_id, standings, pos, match_count):
    if match_count > 0:
        try:
            team_pts = int(standings[standings['team_id'] == team_id]['pts'])
        except:
            return 0
        pos_pts = int(standings.loc[pos, 'pts'])
        
        return (pos_pts - team_pts) / match_count
    else:
        return 0   

In [60]:
def get_rel_pts_diff(team_id, standings, pos):
    try:
        team_pts = float(standings[standings['team_id'] == team_id]['pts'])
    except:
        return 0
    pos_pts = float(standings.loc[pos, 'pts'])

    return (pos_pts - team_pts)

In [61]:
# Create features using the above function.
# N stands for the number of teams in the league.

df['REL_PTS_1_home'] = 0
df['REL_PTS_2_home'] = 0
df['REL_PTS_3_home'] = 0
df['REL_PTS_4_home'] = 0
df['REL_PTS_5_home'] = 0

df['REL_PTS_N-0_home'] = 0
df['REL_PTS_N-1_home'] = 0
df['REL_PTS_N-2_home'] = 0
df['REL_PTS_N-3_home'] = 0
df['REL_PTS_N-4_home'] = 0

df['REL_PTS_1_away'] = 0
df['REL_PTS_2_away'] = 0
df['REL_PTS_3_away'] = 0
df['REL_PTS_4_away'] = 0
df['REL_PTS_5_away'] = 0

df['REL_PTS_N-0_away'] = 0
df['REL_PTS_N-1_away'] = 0
df['REL_PTS_N-2_away'] = 0
df['REL_PTS_N-3_away'] = 0
df['REL_PTS_N-4_away'] = 0

for row, game in df.iterrows():
    for id_name in id_names:
        md = game['RND']
        league = game['league']
        season = game['season']
        team_id = game[id_name]
        team_cnt = game['TEAM_CNT_league']
        
        for i in range(5):
            # Relative points difference to the higher placed teams
            if md > 3:
                cur_standings = get_cur_standings(season, league, row)

                pts_diff_up = get_rel_pts_diff(team_id, cur_standings, i+1)

                # Relative points difference to the lower placed teams
                pts_diff_down = get_rel_pts_diff(team_id, cur_standings, len(cur_standings)-1-i)

                df.loc[row, 'REL_PTS_' + str(i+1) + suffixes[id_name]] = pts_diff_up
                df.loc[row, 'REL_PTS_N-' + str(i) + suffixes[id_name]] = pts_diff_down
            else:
                df.loc[row, 'REL_PTS_' + str(i+1) + suffixes[id_name]] = 0
                df.loc[row, 'REL_PTS_N-' + str(i) + suffixes[id_name]] = 0

In [50]:
# TODO: Das hier drüber testen und funktionabel machen. -> Fixen. 

In [10]:
mc = 38
get_rel_pts_diff(69, get_cur_standings(69, 5, mc, 'I1'), 1, mc)

1.605263157894737

In [59]:
df.sample(10)

,HomeID,AwayID,FTHG,FTAG,FTR,Date,H_WIN_PCT_home,H_DRAW_PCT_home,A_WIN_PCT_home,A_DRAW_PCT_home,...,REL_PTS_1_away,REL_PTS_2_away,REL_PTS_3_away,REL_PTS_4_away,REL_PTS_5_away,REL_PTS_N-0_away,REL_PTS_N-1_away,REL_PTS_N-2_away,REL_PTS_N-3_away,REL_PTS_N-4_away
12489,104,97,3,0,1,2018-02-18,0.421053,0.263158,0.405405,0.135135,...,2,2,1,1,1,0,0,0,1,1
6973,66,78,0,0,0,2012-09-25,0.459459,0.324324,0.184211,0.394737,...,0,0,0,0,-1,-2,-2,-2,-2,-1
7006,72,71,1,0,1,2012-10-21,0.378378,0.297297,0.210526,0.236842,...,2,2,2,2,1,0,0,0,0,0
5507,60,59,3,2,1,2017-04-09,0.333333,0.266667,0.225806,0.193548,...,2,2,1,1,1,1,1,1,1,1
9406,66,70,3,3,0,2019-01-20,0.486486,0.270270,0.307692,0.333333,...,1,1,1,0,0,-1,-1,-1,-1,-1
3338,8,29,4,2,1,2019-03-10,0.692308,0.282051,0.564103,0.282051,...,1,1,1,1,0,-1,-1,0,0,0
9990,118,114,0,0,0,2011-09-11,0.000000,0.000000,0.000000,1.000000,...,0,0,0,0,0,0,0,0,0,0
2496,14,4,2,0,1,2017-01-21,0.405405,0.216216,0.236842,0.368421,...,2,2,2,2,1,0,0,0,0,1
2755,31,13,0,1,2,2017-10-28,0.378378,0.270270,0.263158,0.131579,...,1,1,1,0,0,-1,-1,-1,-1,0
11379,108,97,2,0,1,2015-03-15,0.864865,0.054054,0.605263,0.210526,...,2,2,2,2,1,0,0,0,0,1


In [5]:
df.columns

Index(['HomeID', 'AwayID', 'FTHG', 'FTAG', 'FTR', 'Date', 'H_WIN_PCT_home',
       'H_DRAW_PCT_home', 'A_WIN_PCT_home', 'A_DRAW_PCT_home',
       'H_WIN_PCT_away', 'H_DRAW_PCT_away', 'A_WIN_PCT_away',
       'A_DRAW_PCT_away', 'H_GS_AVG_home', 'H_GC_AVG_home', 'A_GS_AVG_home',
       'A_GC_AVG_home', 'H_GS_AVG_away', 'H_GC_AVG_away', 'A_GS_AVG_away',
       'A_GC_AVG_away', 'H_GS_STD_home', 'H_GC_STD_home', 'A_GS_STD_home',
       'A_GC_STD_home', 'H_GS_STD_away', 'H_GC_STD_away', 'A_GS_STD_away',
       'A_GC_STD_away', 'WIN_PCT_home', 'WIN_PCT_away', 'DRAW_PCT_home',
       'DRAW_PCT_away', 'GS_AVG_home', 'GC_AVG_home', 'GS_AVG_away',
       'GC_AVG_away', 'GS_STD_home', 'GC_STD_home', 'GS_STD_away',
       'GC_STD_away', 'REST_home', 'REST_away', 'H_GS_AVG_league',
       'A_GS_AVG_league', 'H_GS_STD_league', 'A_GS_STD_league',
       'H_WIN_PCT_league', 'DRAW_PCT_league', 'TEAM_CNT_league',
       'GD_STD_league', 'RND_CNT_league', 'H_RTG_home', 'A_RTG_home',
       'H_RTG_away', '

# Save dataframe as .pkl

In [189]:
df.to_pickle('feature_frame.pkl')